In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import isfile, join
import pandas as pd
from scipy.signal import find_peaks


peak_frequency = []
num_peaks = []
run_num = []
# change starting model number using variable "start"

start=1799
num_files = start+201 # num_files includes baseline!

for i in range(start, num_files, 2): # goes through odd files
    #if i == 201:
    #    s = 'baseline'
    #else:
    s = str(i) # changes int i into string s

    path1 = r'../runs_x.01_10/'
    path2 = '/LOGS/history.data'
    file_path = path1 + s + path2

    with open( file_path, 'r') as f:

        info_starts = 0
        data = []
        all_models = []

        for line in f:

            info_starts += 1

            model_info = line.split(" ")    # splits the individual numbers in the lines

            model_info = np.array(list(filter(None, model_info)))[:-1:]
            
            if info_starts == 5: # identifies tau column
                last_column = line.split(" ")
                last_column = np.array(list(filter(None, last_column)))[-2]
                last_column = last_column.astype(int) -1
                
                
            if info_starts == 6:
            
                column_name = {}
                
                for k in range(last_column):
                    column_name[model_info[k]] = k
                    

            if info_starts >= 7:     # This is where we care about the data

                model_elements = model_info.astype(float)

                all_models.append(model_elements)

                num_models = len(all_models)

    # This creates lists for star age and log luminosity

    star_age = []
    log_lum = []
    row_num = []

    # Fills in the above lists with data

    for j in range(num_models):
        row_num.append(j)
        star_age.append(all_models[j][1])
        log_lum.append(all_models[j][column_name['log_L']])


    peaks_x = []
    peaks_y = []
    row_check = []
    x = []
    y = []
    av_lum = []
    total = 0

    # Filling lists with differences of luminosity points 
    
    for j in range(len(log_lum) - 1):     
            x.append(log_lum[j]-log_lum[j-1]) 
            y.append(log_lum[j+1]-log_lum[j]) 
            if log_lum[j] > 0 and log_lum[j+1] - log_lum[j] < .01:
                av_lum.append(log_lum[j])

    # Finds peaks by saying that if luminosities are increasing and then switch to decreasing at a luminosity point
    # then that point is a peak if it's above a certain threshold  
 
    for k in range(len(x)):
        if x[k] > 0 and y[k] < 0 and log_lum[k] > 4.5:
            row_check.append(row_num[k])
            peaks_x.append(star_age[k]*31536000) #converts years to seconds
            peaks_y.append(log_lum[k])
            
   # Checks for extra peaks included by seeing if the peak's row numbers are too close
    # (there's a peak approx. every 35 rows - I use 20 to be cautious)
    # if a peak is less than 20 rows away from the previous, I take the highest one as
    # the peak and discard the other since it is a data anomaly 
    
    row_check = np.array(list(row_check))
    offset = 0
    for m in range(len(row_check)-1):
        if (row_check[m+1] - row_check[m]) < 20:
            if peaks_y[m - offset] < peaks_y[(m-offset)+1]: 
                peaks_y.remove(peaks_y[(m-offset)])        
                peaks_x.remove(peaks_x[(m-offset)])
                offset += 1                                   
            else: 
                peaks_y.remove(peaks_y[(m-offset)+1]) 
                peaks_x.remove(peaks_x[(m-offset)+1])
                offset += 1
                
    # Finds Burst Frequency (if there's more than one peak)
    num_peaks.append(len(peaks_x))
    peak_dist_sum = 0

    if len(peaks_x) == 1:
        print('\n\n\n','Only one peak', '\n\n\n')
        run_num.append(s)
    else:
        for k in range(len(peaks_x)-1):

            peak_dist_sum += (peaks_x[k+1]- peaks_x[k])
            #gets sum distance between all peaks

        burst_frequency = peak_dist_sum / (len(peaks_x) - 1)
       # divides by number of peaks (minus one b/c its an avg. 
        # over distance B E T W E E N peaks)

        peak_frequency.append(burst_frequency)
        run_num.append(s)
        print(burst_frequency, '     run number: ',s, '\n\n')
        print(peaks_x, '\n\n',peaks_y, '\n\n')

peak_frequency = np.array(list(peak_frequency))

8565.942877713398      run number:  1799 


[9932.213292313922, 18590.762044449697, 27181.75207396623, 35766.02869713824, 44312.208531818855, 52843.0906645405, 61375.17771822157, 69911.26267964538, 78473.25030344326, 87025.6991917345] 

 [4.920544174733009, 4.782827896394063, 4.758108433097009, 4.764152181153522, 4.733232437846644, 4.747488498533751, 4.73683360658601, 4.75371060896119, 4.754871902721608, 4.730229364884227] 


8564.94838553103      run number:  1801 


[9932.212456070714, 18554.45700198685, 27119.5579674514, 35717.89432538731, 44206.60320747624, 52742.21378124383, 61349.034939201476, 69931.76831080992, 78540.97515436784, 87014.32260966818, 95581.69631138102] 

 [4.920531771236501, 4.768035161733622, 4.776033383886076, 4.7467836758993505, 4.735188379671032, 4.764873617102855, 4.7518938477468975, 4.738447663521572, 4.747319746970201, 4.753038757745716, 4.745503722070051] 


8583.306831999613      run number:  1803 


[9932.213119520553, 18558.84004614176, 27138.1577049472

8630.774607970243      run number:  1839 


[9932.21383507044, 18586.322602206594, 27285.125860450287, 35891.71012132814, 44509.90351427911, 53106.340119044886, 61826.892179729875, 70416.423921843, 78946.11252298769, 87609.18530680264] 

 [4.920481318891129, 4.806889122932121, 4.775294785965228, 4.772330497988898, 4.762537614929106, 4.7704897695825155, 4.756019259831093, 4.7512455368321485, 4.7733363301919205, 4.7673237970507465] 


8536.704755340088      run number:  1841 


[9932.213623534435, 18563.542606138868, 27131.852952816975, 35660.08211997604, 44245.853143047585, 52854.77413895016, 61333.957520101605, 69931.11937032024, 78419.68791288882, 86768.88486271478, 95299.26117693531] 

 [4.920499862521428, 4.770149781456811, 4.756295580529293, 4.746660542064804, 4.756921221756634, 4.730604466242236, 4.762659027558288, 4.745326670663982, 4.724810546319572, 4.746698342506718, 4.739086462920985] 


8564.460411383005      run number:  1843 


[9932.214101481499, 18558.58120698899, 27104.

8586.673555613304      run number:  1881 


[9932.222991763208, 18602.535045519988, 27215.4145447191, 35794.02967043104, 44360.57265157178, 52925.366058614665, 61493.56520804446, 70067.87338841893, 78627.48167524187, 87212.28499228295] 

 [4.919797365216061, 4.77747516920805, 4.76121927430333, 4.759557404519827, 4.755449641229922, 4.738388106109281, 4.756292117198706, 4.761194796882857, 4.774504566707816, 4.74618116317382] 


8555.484483559034      run number:  1883 


[9932.213875710597, 18590.477961490174, 27151.094062635177, 35717.75813057084, 44228.423249776584, 52771.14018858372, 61352.294746797605, 69922.44340967611, 78315.27265912225, 86931.57422774189] 

 [4.920022090992609, 4.7820109793002485, 4.761908106434713, 4.752669786963496, 4.7330406127573275, 4.741835694889433, 4.760545892797457, 4.72243834205023, 4.742522402804609, 4.768393631900226] 


8607.921184731114      run number:  1885 


[9932.077348804103, 18622.084283766042, 27203.606875391462, 35773.620012850864, 44358.362

8600.925031722963      run number:  1921 


[9932.218504799503, 18604.561229887066, 27216.753889594056, 35885.25987595562, 44366.77139028403, 53041.28627335759, 61569.16366584671, 70156.64326934118, 78790.23677396627, 87340.54379030617] 

 [4.920753206101267, 4.785607255166092, 4.793055222208753, 4.742497302013977, 4.74994542793316, 4.742266112547013, 4.759080703487008, 4.754620044700381, 4.751226060149118, 4.763778862548408] 


8593.231267431236      run number:  1923 


[9932.217555748352, 18613.050771311955, 27224.62928907735, 35811.995333260784, 44461.30720498569, 53132.849706603636, 61645.35805205369, 70274.25636826352, 78811.07730238535, 87271.29896262949] 

 [4.920501939628353, 4.777181988269132, 4.760738781697705, 4.756290929566162, 4.782376743476216, 4.744430927687812, 4.754599419739548, 4.757974869320207, 4.7352317563679875, 4.755482368106042] 


8579.968997919377      run number:  1925 


[9931.540905243093, 18579.4548894593, 27165.458833884462, 35809.6819829301, 44345.44021

8576.064925921404      run number:  1961 


[9932.19017784696, 18599.57337228725, 27122.879432914764, 35749.62563748113, 44353.41807261247, 52847.69273550867, 61464.66520547059, 69926.71518822719, 78541.33885868576, 87116.7745111396] 

 [4.920144342976883, 4.750726145928226, 4.765422961273504, 4.751408183084084, 4.734720554861914, 4.753810445326005, 4.722110673479175, 4.758753613809743, 4.765248281979231, 4.742712551424876] 


8578.300602829364      run number:  1963 


[9932.171240879845, 18574.226056070645, 27136.6064220508, 35710.993704214176, 44201.63511057332, 52832.96453512146, 61502.317672067606, 70161.00448629989, 78718.77716062909, 87221.16911997044, 95715.1772691735] 

 [4.919602359848809, 4.763241831085771, 4.7553642158694025, 4.73659415781377, 4.764887126692546, 4.759708986934301, 4.749234384277199, 4.748952805246892, 4.736787075315198, 4.7331052085248695, 4.75812343558953] 


8586.331794009238      run number:  1965 


[9932.213949336025, 18572.356682993326, 27155.41046721

In [3]:
y = np.linspace(1, 1)
df = pd.DataFrame(data = num_peaks, index = run_num)
# creates a data frame using pandas - essentially tells code below where the data is stored 

writer = pd.ExcelWriter('../num_peaks/num_peaks_.01_10.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=True)
writer.save()

# The above creates, writes in the data, then saves and excel spreadsheet